In [13]:
!pip install pandas

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

invalid syntax (<stdin>, line 1)
  File "<stdin>", line 1
    !pip install pandas
    ^
SyntaxError: invalid syntax



In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, dayofweek, date_format, monotonically_increasing_id
from pyspark.sql.types import  dayofweek, date_format, monotonically_increasing_id

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1676219446898_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

cannot import name 'dayofweek'
Traceback (most recent call last):
ImportError: cannot import name 'dayofweek'



## 1. Get AWS Cluster

### 1.1 Credentials

In [2]:
config = configparser.ConfigParser()
config.read('dl.cfg')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[]

In [3]:
AWS_ACCESS_KEY_ID=config.get("AWS", "AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY=config.get("AWS", "AWS_SECRET_ACCESS_KEY")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

No section: 'AWS'
Traceback (most recent call last):
  File "/usr/lib64/python3.6/configparser.py", line 781, in get
    d = self._unify_values(section, vars)
  File "/usr/lib64/python3.6/configparser.py", line 1141, in _unify_values
    raise NoSectionError(section)
configparser.NoSectionError: No section: 'AWS'



### 1.2 Data Storage

In [4]:
input_data = "s3://sparkify23feb/input_data/"
output_data = "s3://sparkify23feb/output_data/"
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 2. Set Spark Cluster

In [5]:
#.appName("sparkname")
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3. Process Song data 

### 3.1 Get filepath to song data file

In [7]:
song_data = "song_data/*/*/*/*.json" # "song_data/A/A/A/*.json" #
song_path = os.path.join(input_data, song_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 3.2 Read song data file

In [8]:
df_song = spark.read.json(song_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df_song.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

In [10]:
df_song.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

71

In [11]:
df_song.limit(5).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Pandas >= 0.19.2 must be installed; however, it was not found.
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py", line 2075, in toPandas
    require_minimum_pandas_version()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 129, in require_minimum_pandas_version
    "it was not found." % minimum_pandas_version)
ImportError: Pandas >= 0.19.2 must be installed; however, it was not found.



### 3.3 Extract columns to create songs table

**songs** - *songs in music database*
* song_id 
* title
* artist_id 
* year 
* duration

In [14]:
song_table = df_song.select("song_id", "title", "artist_id", "year", "duration").dropDuplicates(["song_id"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 3.4 Write songs table to parquet files partitioned by year and artist

In [15]:
song_table.createOrReplaceTempView('song_table')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
song_outpath = os.path.join(output_data, "song_table.parquet")
song_table.write.partitionBy("year", "artist_id").parquet(song_outpath, 'overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 3.5 Extract columns to create artists table

**artists** - *artists in music database*
* artist_id, 
* name, 
* location, 
* lattitude, 
* longitude

In [17]:
df_song.limit(5).toPandas().columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Pandas >= 0.19.2 must be installed; however, it was not found.
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py", line 2075, in toPandas
    require_minimum_pandas_version()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 129, in require_minimum_pandas_version
    "it was not found." % minimum_pandas_version)
ImportError: Pandas >= 0.19.2 must be installed; however, it was not found.



In [18]:
artisti_columns =[]
artist_table = df_song.select(
    'artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude'\
    ).withColumnRenamed('artist_name', 'name')\
     .withColumnRenamed('artist_location', 'location')\
     .withColumnRenamed('artist_latitude', 'latitude')\
     .withColumnRenamed('artist_longitude', 'longitude').dropDuplicates(["artist_id"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 3.6 Write artists table to parquet files

In [19]:
artist_table.createOrReplaceTempView("artist_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
artists_outpath = os.path.join(output_data, "artist_table.parquet")
artists_table.write.parquet(artist_outpath, "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'artists_table' is not defined
Traceback (most recent call last):
NameError: name 'artists_table' is not defined



## 4 Process log_data

### 4.1. Get filepath to log data file

In [21]:
log_data = "log_data/*.json"
log_path = os.path.join(input_data, log_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 4.2 read log data file

In [22]:
df_log = spark.read.json(log_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
df_log.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

In [24]:
df_log.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8056

In [25]:
df_log.limit(5).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Pandas >= 0.19.2 must be installed; however, it was not found.
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py", line 2075, in toPandas
    require_minimum_pandas_version()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 129, in require_minimum_pandas_version
    "it was not found." % minimum_pandas_version)
ImportError: Pandas >= 0.19.2 must be installed; however, it was not found.



In [26]:
column = "page"
print(df_log.dropDuplicates([column]).select(column).collect())


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(page='Submit Downgrade'), Row(page='Home'), Row(page='Downgrade'), Row(page='Logout'), Row(page='Save Settings'), Row(page='About'), Row(page='Settings'), Row(page='Login'), Row(page='NextSong'), Row(page='Help'), Row(page='Upgrade'), Row(page='Error'), Row(page='Submit Upgrade')]

### 4.3 Filter by actions for song plays

**users** - *users in the app*
* user_id, 
* first_name, 
* last_name, 
* gender, 
* level



In [27]:
df_log = df_log.filter(df_log["page"] == "NextSong")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 4.4 Extract columns for users table    

In [28]:
users_table = df_log.select("userId", "firstName", "lastName", "gender", "level").orderBy("userId").dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
users_table.limit(5).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Pandas >= 0.19.2 must be installed; however, it was not found.
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py", line 2075, in toPandas
    require_minimum_pandas_version()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 129, in require_minimum_pandas_version
    "it was not found." % minimum_pandas_version)
ImportError: Pandas >= 0.19.2 must be installed; however, it was not found.



### 4.5 Write users table to parquet files

In [30]:
users_table.createOrReplaceTempView("users_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
users_outpath = os.path.join(output_data, "users_table.parquet")
users_table.write.parquet(users_outpath, "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 4.6 create timestamp column from original timestamp column
    get_timestamp = udf()
    df = 
**time** - *timestamps of records in songplays broken down into specific units*
* start_time, 
* hour, 
* day, 
* week, 
* month, 
* year, 
* weekday

In [32]:
get_timestamp = udf(lambda x: datetime.fromtimestamp(x/1000).strftime("%Y-%m-%d %H:%M:%S"))
df_time = df_log.withColumn("start_time", get_timestamp("ts"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
df_time.limit(5).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Pandas >= 0.19.2 must be installed; however, it was not found.
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py", line 2075, in toPandas
    require_minimum_pandas_version()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 129, in require_minimum_pandas_version
    "it was not found." % minimum_pandas_version)
ImportError: Pandas >= 0.19.2 must be installed; however, it was not found.



### 4.7 Create datetime column from original timestamp column
    get_datetime = udf()
    df = 

In [34]:
get_datetime = udf(lambda x: datetime.fromtimestamp(x/1000).strftime("%Y-%m-%d"))
df_time = df_log.withColumn("start_time", get_timestamp("ts"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
df_time.limit(5).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Pandas >= 0.19.2 must be installed; however, it was not found.
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py", line 2075, in toPandas
    require_minimum_pandas_version()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 129, in require_minimum_pandas_version
    "it was not found." % minimum_pandas_version)
ImportError: Pandas >= 0.19.2 must be installed; however, it was not found.



### 4.8 Extract columns to create time table
    time_table = 
 start_time,
hour,
day,
week,
month,
year,
weekday

year, month, dayofmonth, hour, weekofyear

In [36]:
time_table = df_time.select("start_time"\
                          ,hour("start_time").alias("hour")\
                          ,dayofmonth("start_time").alias("day")\
                          ,weekofyear("start_time").alias("week")\
                          ,month("start_time").alias("month")\
                          ,year("start_time").alias("year")\
                          ,dayofweek("start_time").alias("weekday")).orderBy("start_time") 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
time_table.limit(5).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Pandas >= 0.19.2 must be installed; however, it was not found.
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py", line 2075, in toPandas
    require_minimum_pandas_version()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 129, in require_minimum_pandas_version
    "it was not found." % minimum_pandas_version)
ImportError: Pandas >= 0.19.2 must be installed; however, it was not found.



### 4.9 Write time table to parquet files partitioned by year and month
    time_table

In [38]:
time_table.createOrReplaceTempView("time_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
time_outpath = os.path.join(output_data, "time_table.parquet")
time_table.write.parquet(time_outpath, "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 45166)
Traceback (most recent call last):
  File "/usr/lib64/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib64/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 266, in handle
    poll(authenticate_and_accum_updates)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 254, in authenticate_and_accum_updates
    received_to

### 4.10 Read in song data to use for songplays table
    
**songplays** - *records in log data associated with song plays i.e. records with page NextSong*
* songplay_id, 
* start_time, 
* user_id, 
* level, 
* song_id, 
* artist_id, 
* session_id, 
* location, 
* user_agent


### 4.11 Extract columns from joined song and log datasets to create songplays table 

In [40]:
cond = [df_log.song==df_song.title, df_log.artist==df_song.artist_name, df_log.length==df_song.duration]
df_join = df_log.join(df_song, cond, "left_outer")\
                .withColumn("songplay_id", monotonically_increasing_id())\
                .withColumn("start_time", get_timestamp("ts"))\
                .withColumnRenamed("userId", "user_id")\
                .withColumnRenamed("sessionId", "session_id")\
                .withColumnRenamed("userAgent", "user_agent")\
                

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
songplays_table = df_join.select("songplay_id", 
                              "start_time", 
                              "user_id", 
                              "level", 
                              "song_id", 
                              "artist_id", 
                              "session_id", 
                              "location", 
                              "user_agent")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 4.12 Write songplays table to parquet files partitioned by year and month
    songplays_tablea

In [42]:
songplays_table.createOrReplaceTempView("songplays_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
songplays_outpath = os.path.join(output_data, "songplays_table.parquet")
songplays_table.write.parquet(songplays_outpath, "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…